In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [26]:
#Read in CSPX.L data
data = pd.read_csv("CSPX_L_historical_data.csv")

#Keep only data from two years ago to now
max_date = pd.Timestamp.now(tz='UTC')
min_date = max_date - pd.DateOffset(years=2)

data['Date'] = pd.to_datetime(data['Date'], utc=True)

data_trimmed = data[(data['Date'] >= min_date) & (data['Date'] <= max_date)]

data_trimmed = data_trimmed.drop(columns=['Dividends', 'Stock Splits','Capital Gains'])

In [27]:
#Create the timelag
#I want yesterdays everything except date
data_trimmed['yesterdays_close'] = data_trimmed['Close'].shift(1)
data_trimmed['yesterdays_open'] = data_trimmed['Open'].shift(1)
data_trimmed['yesterdays_high'] = data_trimmed['High'].shift(1)
data_trimmed['yesterdays_volume'] = data_trimmed['Volume'].shift(1)
data_trimmed['yesterdays_low'] = data_trimmed['Low'].shift(1)
data_trimmed = data_trimmed.iloc[1:]

#Create the true/false flag depending on yesterdays close and todays open
#If todays open is greater than yesterdays close, that's a 1. Otherwise 0
def flag_creator(row):
    if row['Open'] > row['yesterdays_close']:
        return 1
    else:
        return 0

data_trimmed['Success_Flag'] = data_trimmed.apply(flag_creator,axis=1)
data_trimmed = data_trimmed.drop(columns='Open')


In [28]:
#Divide data into test and train sets (using the last two months as the test set)
two_months_ago = max_date - pd.DateOffset(months=2)
data_test = data_trimmed[data_trimmed['Date'] >= two_months_ago]

data_train = data_trimmed[data_trimmed['Date'] <= two_months_ago]

data_train = data_train.drop(columns='Date')
data_test = data_test.drop(columns='Date')

In [30]:
# Example DataFrame `df` with multiple features
# Assume you have the following columns: 'close_t-1', 'RSI', 'MACD', 'Volume', etc.
x_train = data_train[['yesterdays_close','yesterdays_open','yesterdays_volume','yesterdays_high','yesterdays_low']]  # Independent variables
y_train = data_train['Success_Flag']  # Target variable (1 = stock price goes up, 0 = stock price does not go up)

x_test = data_test[['yesterdays_close','yesterdays_open','yesterdays_volume','yesterdays_high','yesterdays_low']]
y_test = data_test['Success_Flag']

# Create and fit the logistic regression model
model = LogisticRegression()
model.fit(x_train, y_train)

# Predict on the test set
y_pred = model.predict(x_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy:.2f}')

# Coefficients and intercept
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Model accuracy: 0.41
Coefficients: [[ 1.21939511e-02 -6.53364759e-02  1.00986741e-06  4.81325202e-03
   4.87920883e-02]]
Intercept: [0.00283658]


C:\Users\thoma\anaconda3\envs\StockData\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
model = GradientBoostingClassifier(n_estimators=1000, learning_rate=1, max_depth=5, random_state=42)
model.fit(x_train, y_train)

# Make predictions
y_pred = model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy:.2f}')

# Model performance metrics (optional)
print(f'Feature importances: {model.feature_importances_}')

Model accuracy: 0.56
Feature importances: [0.15083268 0.18353088 0.3070979  0.24487278 0.11366576]
